Chinook Music Store Data

    Querying a database file with 12 tables and well over 100 columns. 

>This project was completed as part of a SQL Course in the DataQuest.io Certification program showing more complex queries and the requirements to complete each of them. Although the answers are filled in here the program used to run these queries versus a .db file was 'JetBrains DataGrip' with SQLite as the SQL dialect. 

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

>The ask was to show all tables or views associated with this .db file. 

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");


Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The ask was to create a query which would return:
* Each Genre
* Number of Tracks sold per Genre: 
<br>-In Absolute Numbers
<br>-In Percentages
<br>


In [8]:
%%sql

WITH
full_join AS
(SELECT g2.name genre, cast(count(t2.track_id) as float) track_count
FROM customer c
INNER JOIN invoice i2 on c.customer_id = i2.customer_id
INNER JOIN invoice_line il on i2.invoice_id = il.invoice_id
INNER JOIN track t2 on il.track_id = t2.track_id
INNER JOIN genre g2 on t2.genre_id = g2.genre_id
WHERE c.country='USA'
GROUP BY g2.name),

for_count AS
(SELECT count(*)
FROM customer c
INNER JOIN invoice i2 on c.customer_id = i2.customer_id
INNER JOIN invoice_line il on i2.invoice_id = il.invoice_id
INNER JOIN track t2 on il.track_id = t2.track_id
INNER JOIN genre g2 on t2.genre_id = g2.genre_id
WHERE c.country='USA')

select full_join.genre, full_join.track_count, round(full_join.track_count/(select * from for_count)*100,3) as track_percentage
from full_join
order by track_percentage desc

Done.


genre,track_count,track_percentage
Rock,561.0,53.378
Alternative & Punk,130.0,12.369
Metal,124.0,11.798
R&B/Soul,53.0,5.043
Blues,36.0,3.425
Alternative,35.0,3.33
Latin,22.0,2.093
Pop,22.0,2.093
Hip Hop/Rap,20.0,1.903
Jazz,14.0,1.332


> The goal was to see which of four new tracks should be added with genres: Hip-Hop, Punk, Pop, Blues using top grossing genres as the decision making marker.

Given these four choices and the chart above it is sensible to choose the <b>Punk, Pop, and Blues</b> songs for the store. 

The ask was to run a query which would return the full name of 'Sales Support Agents' and the dollar amounts they each managed on the records store.

In [6]:
%%sql

/* total dollar amount per sales agent */
WITH
rep_table AS (
SELECT e.first_name||' '||e.last_name rep_name, e.title, round(sum(i.total), 2) sales
FROM employee e
INNER JOIN customer c on e.employee_id = c.support_rep_id
INNER JOIN invoice i on c.customer_id = i.customer_id
WHERE e.title='Sales Support Agent'
GROUP BY rep_name
ORDER BY sales desc
)

select * from rep_table

Done.


rep_name,title,sales
Jane Peacock,Sales Support Agent,1731.51
Margaret Park,Sales Support Agent,1584.0
Steve Johnson,Sales Support Agent,1393.92


The ask is to run a query where for each country it includes:
<br> -the total number of customers
<br> -total value of sales
<br> -average value of sales per customer
<br> -average order value
<br>
<br> There was a column which is the individual ID for the invoice assigned to each row. I summed up the amount for each invoice and created the order total in this manner. 

In [11]:
%%sql
WITH
     intuit AS (
SELECT c.customer_id customer_id, 
        c.country country, 
         count(DISTINCT c.customer_id) num_customers, 
         sum(i.total) total_sales
FROM customer c
LEFT JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY c.country
ORDER BY  num_customers DESC),

inv_value AS (
    SELECT c2.customer_id customer_id, 
    invoice_id, sum(inv.total) invoice_total
    FROM invoice inv
             LEFT JOIN customer c2 ON inv.customer_id = c2.customer_id
    GROUP BY invoice_id
    ORDER BY invoice_total DESC
),

inv_average AS (
SELECT c3.customer_id customer_id, 
    sum(total) sale_per_customer
FROM customer c3
LEFT JOIN invoice i2 ON c3.customer_id = i2.customer_id
GROUP BY c3.customer_id


)

SELECT country,
        intuit.num_customers,
        round(intuit.total_sales,2) total_sales,
        round(avg(inv_average.sale_per_customer), 2) avg_per_customer,
        round(avg(inv_value.invoice_total), 2) average_invoice
FROM inv_value
LEFT JOIN intuit ON intuit.customer_id=inv_value.customer_id
LEFT JOIN inv_average ON intuit.customer_id=inv_average.customer_id
GROUP BY country
HAVING country IS NOT NULL
ORDER BY num_customers DESC 


Done.


country,num_customers,total_sales,avg_per_customer,average_invoice
USA,13,1040.49,72.27,7.23
Canada,8,535.59,75.24,6.27
Brazil,5,427.68,106.92,7.13
France,5,389.07,73.26,6.1
Germany,4,334.62,73.26,8.14
United Kingdom,3,245.52,79.2,8.8
Czech Republic,2,273.24,128.7,10.73
India,2,183.15,71.28,8.91
Portugal,2,185.13,82.17,5.14
Argentina,1,39.6,39.6,7.92


The ask for the for the cell below was to run a query which would inform us on an approximate split between buying complete albums on the store and buying individual tracks from those albums.

In [32]:
%%sql
WITH album_main as (
SELECT  a.album_id album_id,  count( distinct t.track_id) track_id_num
FROM album a
LEFT JOIN track t on a.album_id = t.album_id
LEFT JOIN invoice_line il on t.track_id = il.track_id
LEFT JOIN invoice i on il.invoice_id = i.invoice_id
GROUP BY a.album_id

),

invoiced_as AS (
SELECT i.invoice_id invoice_id, a.album_id album_id, count(distinct t.track_id) tracks_per_invoice
FROM album a
LEFT JOIN track t on a.album_id = t.album_id
LEFT JOIN invoice_line il on t.track_id = il.track_id
LEFT JOIN invoice i on il.invoice_id = i.invoice_id
GROUP BY i.invoice_id

order by i.invoice_id desc
),

bought_it AS (

select invoiced_as.invoice_id, invoiced_as.album_id from_album, album_main.album_id album_id, album_main.track_id_num tracks_on_album, invoiced_as.tracks_per_invoice,
       case when (tracks_per_invoice/album_main.track_id_num) - 1 > 0 then 'Manually Added Tracks'
            when tracks_per_invoice/album_main.track_id_num < 1.0 then 'Album Not Bought'
            when tracks_per_invoice/album_main.track_id_num = 1.0 then 'Album Bought'
            END as bought_or_not
from invoiced_as
left join album_main on album_main.album_id=invoiced_as.album_id)

select bought_or_not, round(cast(count(bought_or_not) as float)/(select count(*) from bought_it)*100,2) bought_percent
from bought_it
group by bought_or_not

Done.


bought_or_not,bought_percent
Album Bought,19.84
Album Not Bought,66.67
Manually Added Tracks,13.5


After running the query for analysis it shows that if doing away with the ability to purchase a complete album the loss in sales would be significant. 
<br>
<br>The analysis was conducted by looking at the invoices, counting how many tracks were in the invoice, comparing that to the number of tracks in the albums we have listed. 

<br> <b>This gives rise to three scenarios:</b>
>1. The amount of tracks is greater than the number in the album which means the person added individual tracks manually. This was one of the options customers had when purchasing as well. 
>2. The amount of tracks purchased in the invoice was below the amount of tracks in the album, which naturally means the complete album was not purchased. 
>3. The amount of tracks in the invoice are the same which would mean the complete album was purchased. 